# Set up

In [1]:
%%capture
!pip install langchain
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install pypdf
!pip install pdfminer.six
!pip install -q -U bitsandbytes
!pip install openai
!pip install tiktoken

In [2]:
from huggingface_hub import login
token = "hf_token"
login(token=token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate,  LLMChain

from transformers import AutoTokenizer
import transformers
import torch
import re

# Data preparation

## Data

### Kenya

In [4]:
loader =  PyPDFLoader("/content/drive/MyDrive/aphrc_llm/data/Kenya.pdf")
pages = loader.load_and_split()

In [5]:
part_1 = pages[5].page_content + pages[6].page_content + pages[7].page_content + pages[8].page_content + pages[9].page_content
part_2 =  pages[9].page_content + pages[10].page_content + pages[11].page_content + pages[12].page_content + pages[13].page_content + pages[14].page_content
part_3 = pages[14].page_content + pages[15].page_content + pages[16].page_content + pages[17].page_content
part_4 = pages[17].page_content + pages[18].page_content + pages[19].page_content + pages[20].page_content + pages[21].page_content + pages[22].page_content + pages[23].page_content + pages[24].page_content + pages[25].page_content + pages[26].page_content + pages[27].page_content + pages[28].page_content + pages[29].page_content + pages[30].page_content
part_5 = pages[30].page_content + pages[31].page_content
part_6 = pages[31].page_content + pages[32].page_content + pages[33].page_content
part_7 = pages[33].page_content + pages[34].page_content
part_8 = pages[34].page_content + pages[35].page_content +  pages[36].page_content + pages[37].page_content + pages[38].page_content +  pages[39].page_content
part_9 = pages[39].page_content + pages[40].page_content
part_10 = pages[40].page_content + pages[41].page_content
part_11 = pages[41].page_content + pages[42].page_content + pages[43].page_content

In [6]:
def extract_part(part):
  pattern = r"PART .*?(?=PART )"
  match = re.search(pattern, part, flags=re.DOTALL)
  return match.group()

def extract_part_11(part):
  pattern = r"PART .*?(?=amendments.)"
  match = re.search(pattern, part, flags=re.DOTALL)
  return match.group()

In [7]:
part_1 = extract_part(part_1)
part_2 = extract_part(part_2)
part_3 = extract_part(part_3)
part_4 = extract_part(part_4)
part_5 = extract_part(part_5)
part_6 = extract_part(part_6)
part_7 = extract_part(part_7)
part_8 = extract_part(part_8)
part_9 = extract_part(part_9)
part_10 = extract_part(part_10)
part_11 = extract_part_11(part_11)


In [8]:
kenya = {}
for i in range(1, 12):
    kenya['part_' + str(i)] = globals()['part_' + str(i)]


In [9]:
ke_full_doc = part_1 + part_2 + part_3 + part_4 + part_5 + part_6 + part_7 + part_8 + part_9 + part_10 + part_11

## Rwanda

In [10]:
loader =  PyPDFLoader("/content/drive/MyDrive/aphrc_llm/data/Rwanda.pdf")
pages = loader.load_and_split()

In [11]:
chapter_1 = pages[5].page_content + pages[6].page_content + pages[7].page_content + pages[8].page_content + pages[9].page_content + pages[10].page_content
chapter_2 = pages[11].page_content + pages[12].page_content + pages[13].page_content + pages[14].page_content + pages[15].page_content + pages[16].page_content + pages[17].page_content + pages[18].page_content + pages[19].page_content + pages[20].page_content
chapter_3 = pages[21].page_content + pages[22].page_content + pages[23].page_content
chapter_4 = pages[24].page_content + pages[25].page_content + pages[26].page_content + pages[27].page_content + pages[28].page_content
chapter_5 = pages[29].page_content + pages[30].page_content + pages[31].page_content + pages[32].page_content + pages[33].page_content + pages[34].page_content + pages[35].page_content + pages[36].page_content
chapter_6 = pages[37].page_content + pages[38].page_content + pages[39].page_content
chapter_7 = pages[40].page_content + pages[41].page_content + pages[42].page_content + pages[43].page_content + pages[44].page_content + pages[45].page_content
chapter_8 = pages[46].page_content + pages[47].page_content
chapter_9 = pages[48].page_content + pages[49].page_content + pages[50].page_content
chapter_10 = pages[51].page_content



In [12]:
rw_full_doc = chapter_1 + chapter_2 + chapter_3 + chapter_4 + chapter_5 + chapter_6 + chapter_7 + chapter_8 + chapter_9 + chapter_10

In [13]:
rwanda = {}
for i in range(1, 11):
    rwanda['chapter_' + str(i)] = globals()['chapter_' + str(i)]

# Uganda

In [14]:
loader =  PyPDFLoader("/content/drive/MyDrive/aphrc_llm/data/Uganda.pdf")
pages = loader.load_and_split()

In [15]:
ug_part_1 = pages[4].page_content + pages[5].page_content + pages[6].page_content + pages[7].page_content
ug_part_2 =  pages[8].page_content + pages[9].page_content + pages[10].page_content
ug_part_3 = pages[10].page_content + pages[11].page_content + pages[12].page_content + pages[13].page_content + pages[14].page_content+ pages[15].page_content+ pages[16].page_content+ pages[17].page_content+ pages[18].page_content+ pages[19].page_content
ug_part_4 = pages[19].page_content + pages[20].page_content + pages[21].page_content
ug_part_5 = pages[21].page_content + pages[22].page_content + pages[23].page_content + pages[24].page_content + pages[25].page_content + pages[26].page_content
ug_part_6 = pages[26].page_content + pages[27].page_content
ug_part_7 = pages[27].page_content + pages[28].page_content
ug_part_8 = pages[28].page_content + pages[29].page_content

In [16]:
ug_part_1 = ug_part_1[610:]
ug_part_2 = extract_part(ug_part_2)
ug_part_3 = extract_part(ug_part_3)
ug_part_4 = extract_part(ug_part_4)
ug_part_5 = extract_part(ug_part_5)
ug_part_6 = extract_part(ug_part_6)
ug_part_7 = extract_part(ug_part_7)
ug_part_8 = ug_part_8[200:]

In [17]:
uganda = {}
for i in range(1, 9):
    uganda['ug_part_' + str(i)] = globals()['ug_part_' + str(i)]


In [18]:
ug_full_doc = ug_part_1 + ug_part_2 + ug_part_3 + ug_part_4 + ug_part_5 + ug_part_6 + ug_part_7 + ug_part_8

### Preprocessing

In [19]:
country = "Kenya"
section = "5"

In [20]:
if country == "Kenya":
  if section == None:
    Text = ke_full_doc
  else:
      Text = kenya[f"part_{section}"]
elif country == "Rwanda":
  if section == None:
    Text = rw_full_doc
  else:
      Text = rwanda[f"chapter_{section}"]
elif country == "Uganda":
  if section == None:
    Text = ug_full_doc
  else:
      Text = uganda[f"ug_part_{section}"]


In [21]:
def split_text_into_chunks(text, chunk_size):
    """Split the input text into chunks of a given size."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [22]:
if len(Text) > 4000:
  chunk_size = 3000
  Text = split_text_into_chunks(Text, chunk_size)

# Model

## OpenAI

In [23]:
import openai

openai.api_key = "openai_token"

In [24]:
model_name = "gpt-3.5-turbo"

In [25]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, openai_api_key=openai.api_key, model_name=model_name)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [26]:
template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in less words in a paragraph which covers the key points of the text.
              ```{text}```
              PARAGRAPH SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [27]:
Text = """
PART X —PROVISIONS ON DELEGATED POWERS 71. The Cabinet Secretary may, make regulations generally for giving effect to this Act, and for prescribing anything required or necessary to be prescribed by or under this Act. (2) Without prejudice to the generality of subsection (1), regulations made under that subsection may provide for— (a) the requirements which are imposed on a data controller or data processor when processing personal data; (b) mechanisms of conducting certification program; (c) the contents which a notice or registration by a data controller or data processor should contain; (d) information to be provided to a data subject and how such information shall be provided; (e) the levying of fees and taking of charges; Regulations. 941 2019 Data Protection (f) the measures to safeguard a data subject's rights, freedoms and legitimate interests; (g) the processing of data through a data server or data centre in Kenya; (h) issuing and approval of codes of practice and guidelines; or (i) any other matter that the Cabinet Secretary may deem fit. (3) For the purposes of Article 94 (6) of the Constitution — (a) the purpose and objective of the delegation under this section is to enable the Cabinet Secretary to make regulations for better carrying into effect the provisions of this Act; (b) the authority of the Cabinet Secretary to make regulations under this Act will be limited to bringing into effect the provisions of this Act and fulfilment of the objectives specified under this section. (4) The principles and standards applicable to the delegated power referred to under this Act are those found in— (a) the Statutory Instruments Act, 2013; (b) the Interpretation and General Provisions Act; (c) the general rules of international law as specified under Article 2 (5) of the Constitution; and (d) any treaty and convention ratified by Kenya under Article 2 (6) of the Constitution.


"""

In [29]:
if type(Text) == str:
  summary = llm_chain.run({"country":country, "text": Text})
else:
  summary = ""
  for i in range(len(Text)):
    summary = summary + llm_chain.run({"country":country, "text": Text[i]})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [30]:
summary

"The Cabinet Secretary has the authority to make regulations for the implementation of the Data Protection Act. These regulations can cover various aspects such as the requirements for data controllers and processors, certification programs, contents of notices or registrations, information provided to data subjects, fees and charges, safeguarding data subjects' rights, processing data in Kenya, issuing codes of practice and guidelines, and any other matters deemed necessary. The purpose of this delegation of power is to effectively enforce the provisions of the Act, and the Cabinet Secretary's authority is limited to fulfilling the objectives of the Act. The principles and standards applicable to these regulations are outlined in the Statutory Instruments Act, Interpretation and General Provisions Act, general rules of international law, and any ratified treaties or conventions."